In [1]:
# Padrão
import pandas as pd 
import numpy as np 
import lightgbm as lgb

# Visualization
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_style('darkgrid')

# Others
import openpyxl
import xlrd
import os
import statistics
from math import isnan

# Metrics from Sklearn
from sklearn import metrics
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             precision_score, 
                             recall_score, 
                             confusion_matrix, 
                             auc,
                             roc_curve, 
                             roc_auc_score, 
                             f1_score, auc)
from sklearn.pipeline import make_pipeline

In [2]:
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning) 
warnings.filterwarnings("ignore", category = FutureWarning) 
warnings.filterwarnings("ignore", category = UserWarning) 

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
path_in = '~\\Documents\\PRJ_OTIM_PRICING\\data\\processed\\'
#path_out = '~\\Documents\\PRJ_OTIM_PRICING\\data\\processed\\'

In [5]:
name_file_df = 'dataframe_processed.csv'
file_in = os.path.join(path_in, name_file_df)

In [6]:
df_risk = pd.read_csv(file_in, sep=",", encoding='UTF-8', low_memory=False)
df_risk.shape

(642248, 33)

In [7]:
df_risk.isnull().sum().sum()

0

In [8]:
df_risk.describe().T

,count,mean,std,min,25%,50%,75%,max
id,642248.000,6347226.380,3653107.015,58.000,3471258.750,5527281.000,8649916.250,14435572.000
Faixa_Hora_Contrat,642248.000,5.849,0.541,1.000,6.000,6.000,6.000,7.000
Faixa_Remun_Dezem_(SM),642248.000,3.797,2.566,0.000,3.000,4.000,5.000,12.000
Escolaridade_após_2005,642248.000,6.463,1.599,1.000,6.000,7.000,7.000,11.000
Qtd_Hora_Contr,642248.000,42.439,5.700,0.000,44.000,44.000,44.000,44.000
Idade,642248.000,36.032,11.174,14.000,27.000,35.000,44.000,94.000
Qtd_Dias_Afastamento,642248.000,7.427,27.768,0.000,0.000,0.000,2.000,365.000
Raça_Cor,642248.000,3.720,1.586,1.000,2.000,3.000,5.000,7.000
Sexo_Trabalhador,642248.000,1.309,0.462,1.000,1.000,1.000,2.000,2.000
Tamanho_Estabelecimento,642248.000,7.055,2.466,1.000,5.000,7.000,9.000,10.000


In [9]:
def unique_values(df):
    all_cols = df.columns
    feature_name = []
    num_unique_val = []
    name_unique_val = []
    datatype = []
    
    for col in all_cols:
        feature_name.append(col)
        num_unique_val.append(df[col].nunique())
        name_unique_val.append(df[col].unique())
        datatype.append(df[col].dtype)
    return pd.DataFrame({'Feature Name': feature_name, 'Number of Unique Value': num_unique_val, 'Name of Unique Values': name_unique_val,'datatype': datatype})

In [10]:
unique_values(df_risk)

,Feature Name,Number of Unique Value,Name of Unique Values,datatype
0,id,642248,"[2523503, 2523514, 2523519, 2523536, 2523571, ...",int64
1,Faixa_Hora_Contrat,7,"[6, 1, 5, 3, 4, 7, 2]",int64
2,Faixa_Remun_Dezem_(SM),13,"[3, 5, 2, 4, 1, 6, 7, 9, 8, 10, 11, 12, 0]",int64
3,Escolaridade_após_2005,11,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",int64
4,Qtd_Hora_Contr,45,"[44, 1, 36, 20, 22, 40, 38, 42, 39, 43, 0, 34,...",int64
5,Idade,77,"[50, 61, 68, 34, 49, 27, 58, 47, 25, 60, 44, 4...",int64
6,Ind_Simples,2,"[True, False]",bool
7,Ind_Portador_Defic,2,"[False, True]",bool
8,Qtd_Dias_Afastamento,366,"[0, 20, 62, 25, 81, 5, 7, 4, 40, 3, 10, 58, 9,...",int64
9,Raça_Cor,7,"[3, 2, 5, 6, 4, 1, 7]",int64


In [11]:
interger_teste = ['is_cotacao', 'is_apolice', 'is_sinistro']   
for  Names in interger_teste:  
    print(df_risk[Names].value_counts())

is_cotacao
0    537414
1    104834
Name: count, dtype: int64
is_apolice
0    598704
1     43544
Name: count, dtype: int64
is_sinistro
0.000    636857
1.000      5391
Name: count, dtype: int64


In [12]:
df_risk = df_risk[(df_risk['is_cotacao'] == 1)].reset_index(drop = True)
df_risk.shape

(104834, 33)

In [13]:
interger_teste = ['is_cotacao', 'is_apolice', 'is_sinistro']   
for  Names in interger_teste:  
    print(df_risk[Names].value_counts())

is_cotacao
1    104834
Name: count, dtype: int64
is_apolice
0    61290
1    43544
Name: count, dtype: int64
is_sinistro
0.000    99443
1.000     5391
Name: count, dtype: int64


In [14]:
df_risk.head()

,id,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,corretora,Sexo_Trabalhador,Tamanho_Estabelecimento,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,premio_input,Tempo_Emprego,Importancia_Segurada,is_cotacao,is_apolice,is_sinistro,Vlr_Sinistro,Comission,Empresa_contrante,cidades,premio_comercial,vlr_comission,premio_puro,Desc_Faixa_Etária
0,2533200,6,3,1,44,65,True,False,0,2,corretora_2222600,1,6,1,1,10,IBGE_Subsetor_10,False,False,1231.930,110.400,14902.780,1,0,0.000,0.000,0.250,EC_5192,mg_varginha,1141.010,285.250,513.460,65_anos_ou_mais
1,2680702,6,4,3,44,63,True,False,0,2,corretora_2451200,1,5,1,1,10,IBGE_Subsetor_3,False,False,1689.710,322.700,20492.080,1,0,0.000,0.000,0.250,EC_8214,mg_cláudio,1565.010,391.250,704.250,50_A_64 anos
2,2696958,6,3,3,44,56,True,False,0,2,corretora_2451200,1,5,1,1,10,IBGE_Subsetor_3,False,False,1580.220,401.800,19616.180,1,0,0.000,0.000,0.250,EC_7222,mg_cláudio,1463.600,365.900,658.620,50_A_64 anos
3,2841900,6,4,4,44,71,True,False,0,2,corretora_2451200,1,5,1,1,10,IBGE_Subsetor_3,False,False,1689.710,461.800,20382.600,1,0,0.000,0.000,0.250,EC_7223,mg_cláudio,1565.010,391.250,704.250,65_anos_ou_mais
4,2841901,6,4,4,44,56,True,False,0,5,corretora_2451200,1,5,1,1,10,IBGE_Subsetor_3,False,False,1689.710,290.200,20477.420,1,0,0.000,0.000,0.250,EC_7201,mg_cláudio,1565.010,391.250,704.250,50_A_64 anos


In [15]:
df_risk.columns

Index(['id', 'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)',
       'Escolaridade_após_2005', 'Qtd_Hora_Contr', 'Idade', 'Ind_Simples',
       'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Raça_Cor', 'corretora',
       'Sexo_Trabalhador', 'Tamanho_Estabelecimento', 'Tipo_Admissão',
       'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente',
       'Ind_Trab_Parcial', 'premio_input', 'Tempo_Emprego',
       'Importancia_Segurada', 'is_cotacao', 'is_apolice', 'is_sinistro',
       'Vlr_Sinistro', 'Comission', 'Empresa_contrante', 'cidades',
       'premio_comercial', 'vlr_comission', 'premio_puro',
       'Desc_Faixa_Etária'],
      dtype='object')

In [44]:
features_selected_risk = df_risk[['id', 
                                  'Faixa_Etária', 'Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 
                                  'Qtd_Hora_Contr', 'Idade', 'Ind_Simples', 'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 
                                  'Raça_Cor', 'corretora', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento', 'Tipo_Admissão',
                                  'Tipo_Defic', 'Tipo_Vínculo', 'IBGE_Subsetor', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 
                                  'Tempo_Emprego', 'Importancia_Segurada', 'Vlr_Sinistro', 'CBO', 'cidades', 'premio_input',
                                  'premio_puro', 'premio_comercial', 'vlr_comission', 'is_apolice']]
features_selected_risk.shape

(269590, 30)

In [45]:
features_selected_risk = features_selected_risk.set_index(['id'])
features_selected_risk.shape

(269590, 29)

In [60]:
features_selected_risk['is_apolice'].value_counts(normalize=True)*100

is_apolice
0   100.000
Name: proportion, dtype: float64

In [46]:
X = features_selected_risk.drop(columns='is_apolice')
y = features_selected_risk.loc[:,'is_apolice']

In [47]:
print(X.shape)
print(y.shape)

(269590, 28)
(269590,)


In [48]:
X.head()

,Faixa_Etária,Faixa_Hora_Contrat,Faixa_Remun_Dezem_(SM),Escolaridade_após_2005,Qtd_Hora_Contr,Idade,Ind_Simples,Ind_Portador_Defic,Qtd_Dias_Afastamento,Raça_Cor,corretora,Sexo_Trabalhador,Tamanho_Estabelecimento,Tipo_Admissão,Tipo_Defic,Tipo_Vínculo,IBGE_Subsetor,Ind_Trab_Intermitente,Ind_Trab_Parcial,Tempo_Emprego,Importancia_Segurada,Vlr_Sinistro,CBO,cidades,premio_input,premio_puro,premio_comercial,vlr_comission
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
236,6,6,3,1,44,40,True,False,0,3,3101200,1,5,3,1,10,7,False,False,1.200,1785.290,0.000,7841.0,mg_guiricema,1373.300,445.180,1271.950,445.180
308,6,6,3,1,44,48,False,False,5,5,1071600,1,10,1,1,20,13,False,False,246.900,13408.900,0.000,6220.0,mg_urucânia,1193.520,607.990,1105.440,165.820
418,5,6,4,1,44,32,False,True,17,2,3101200,1,9,1,5,10,7,False,False,76.400,18612.850,0.000,7842.0,mg_carmodocajuru,1800.190,917.030,1667.340,250.100
419,6,6,4,1,44,45,False,False,1,5,1071600,1,10,1,1,20,13,False,False,76.200,22852.360,0.000,6221.0,mg_iturama,1745.930,889.390,1617.080,242.560
459,7,5,4,1,36,54,False,False,45,2,2411300,1,8,3,1,10,3,False,False,9.800,12951.190,0.000,6326.0,mg_setelagoas,1973.200,822.410,1827.580,456.890


In [49]:
y.head()

id
236    0
308    0
418    0
419    0
459    0
Name: is_apolice, dtype: int64

In [54]:
# categorical columns without an order :

a = unique_values(df_risk)
a = a[(a['datatype'] == 'object')]
a['Feature Name'].tolist()

['Faixa_Etária', 'corretora', 'IBGE_Subsetor', 'CBO', 'cidades']

In [55]:
# categorical columns without an order :

nominal_columns = ['Faixa_Etária', 'corretora', 'IBGE_Subsetor', 'CBO', 'cidades']
nominal_columns

['Faixa_Etária', 'corretora', 'IBGE_Subsetor', 'CBO', 'cidades']

In [57]:
# categorical columns with an order (probably) :

a = unique_values(df_risk)
a = a[(a['datatype'] == 'int64')]
a['Feature Name'].tolist()

['id',
 'Faixa_Hora_Contrat',
 'Faixa_Remun_Dezem_(SM)',
 'Escolaridade_após_2005',
 'Qtd_Hora_Contr',
 'Idade',
 'Qtd_Dias_Afastamento',
 'Raça_Cor',
 'Sexo_Trabalhador',
 'Tamanho_Estabelecimento',
 'Tipo_Admissão',
 'Tipo_Defic',
 'Tipo_Vínculo',
 'is_cotacao',
 'is_apolice',
 'is_sinistro']

In [58]:
# categorical columns with an order (probably) :

ordinal_columns = ['Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Raça_Cor', 'Sexo_Trabalhador',
                   'Tamanho_Estabelecimento', 'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo']
ordinal_columns

['Faixa_Hora_Contrat',
 'Faixa_Remun_Dezem_(SM)',
 'Escolaridade_após_2005',
 'Raça_Cor',
 'Sexo_Trabalhador',
 'Tamanho_Estabelecimento',
 'Tipo_Admissão',
 'Tipo_Defic',
 'Tipo_Vínculo']

In [59]:
categorical_cols = ordinal_columns + nominal_columns
print('total categorical columns in feature:',len(categorical_cols),"\n\n",categorical_cols)

print("="*60)
print()
numerical_cols = [i for i in X.columns if i not in categorical_cols]
print('total numerical columns in feature:',len(numerical_cols),"\n\n",numerical_cols)

total categorical columns in feature: 14 

 ['Faixa_Hora_Contrat', 'Faixa_Remun_Dezem_(SM)', 'Escolaridade_após_2005', 'Raça_Cor', 'Sexo_Trabalhador', 'Tamanho_Estabelecimento', 'Tipo_Admissão', 'Tipo_Defic', 'Tipo_Vínculo', 'Faixa_Etária', 'corretora', 'IBGE_Subsetor', 'CBO', 'cidades']

total numerical columns in feature: 14 

 ['Qtd_Hora_Contr', 'Idade', 'Ind_Simples', 'Ind_Portador_Defic', 'Qtd_Dias_Afastamento', 'Ind_Trab_Intermitente', 'Ind_Trab_Parcial', 'Tempo_Emprego', 'Importancia_Segurada', 'Vlr_Sinistro', 'premio_input', 'premio_puro', 'premio_comercial', 'vlr_comission']
